# Midterm

## Imports

In [1]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text


In [ ]:
host_name = "Midterm"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"
src_dbname = "chinook"
dst_dbname = "chinook_dw"


In [ ]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str)
with sqlEngine.connect() as connection:
    connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
    connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))


In [ ]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str)
    with sqlEngine.connect() as connection:
        dframe = pd.read_sql(sql_query, connection)
    return dframe

def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str)
    with sqlEngine.connect() as connection:
        if db_operation == "insert":
            df.to_sql(table_name, con=connection, index=False, if_exists='replace')
            connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
        elif db_operation == "update":
            df.to_sql(table_name, con=connection, index=False, if_exists='append')
